In [1]:
import pandas as pd 
import numpy as np
import os as os
import sklearn as sklearn

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'
if os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = '/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM'
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'



In [3]:
# Read base_prompt files from the prompts folder

base_prompt_files = ['p1v1', 'p2v1', 'p3v1']

base_prompts = []

for index, prompt_file in enumerate(base_prompt_files):
    print(index)
    print(prompt_file)
    # Read the files as a list and save it to base_prompts
    file_path = path_equity_precision_llm_repo + "/prompts/" + prompt_file + '.txt'


    with open(file_path, "r") as file:
        # Create a base_prompts list of the three prompts saved as an index 
        # base_prompts[0]: p1v1.txt
        # base_prompts[1]: p2v1.txt
        # base_prompts[2]: p3v1.txt
        base_prompts.append(file.read()) 

print(base_prompts)
    



0
p1v1
1
p2v1
2
p3v1
['I am going to outline inclusion criteria for four categories: diabetes, precision medicine, correct source population, and primary study. \n\nPlease wait for me to prompt you on what to do based on these criteria.\n', 'Here are the inclusion criteria:  \n\nDIABETES: Do not exclude any type of diabetes or prediabetes. \n\nPRECISION MEDICINE: Precision medicine is an assessment of genetic or metabolic state to guide preventive and therapeutic decisions in humans. Exclude epidemiological studies using traditional biomarkers only, focusing on omics (genomics, metabolomics, proteomics, lipidomics etc.) or multi-omics studies. \n\nCORRECT SOURCE POPULATION: The source population is correct if the population based on the prompt and the studyâ€™s geographic source population are the same. The source population refers to humans specifically, excluding plants and therapeutics that may originate in the GBD region. The options are Central Asia (CA), Central and Eastern Europ

Read the PMID's abstract, title and mesh terms

In [29]:
import re as re
def prompt_generator(PMID, base_prompt_list, excel_path):
    print(PMID)

    # Step 1: Read the excel sheet
    df = pd.read_excel(excel_path, sheet_name='Training Data')

    # Step 2: Filter the excel sheet for the PMID, and create 3 variables: title, abstract, mesh
    record = df[df['PMID'] == PMID].iloc[0]
    title = record['Title']
    abstract = record['Abstract']
    mesh = record['MeSH']
    print(mesh)

    # Step 3: Generate the prompt based on the PMID and source_population
    prompt_template = base_prompt_list[2] 

        # For base_prompt_list[2] (i.e. p3vN.txt) 
        # Replace <INSERT TITLE> with the title (title)
        # Replace <INSERT ABSTRACT> with the abstract (abstract)
        # Replace <INSERT MESH TERMS> with the MeSH terms (mesh)

        # Use regular expressions to replace the text in the base prompt with the title, abstract, and MeSH terms
    prompt_new = re.sub(r"<INSERT TITLE>", title, prompt_template)
    prompt_new = re.sub(r"<INSERT ABSTRACT>", abstract, prompt_new)
    if pd.notna(mesh):
        print("Mesh is not null")
        prompt_new = re.sub(r"<INSERT MESH TERMS>", mesh, prompt_new)
        # Step 4: Return the prompt
    else:
        prompt_new = re.sub(r"<INSERT MESH TERMS>", "", prompt_new)
    
    # Handling exceptions
    prompt_new = re.sub(r"\u03b2", "beta", prompt_new)
    prompt_new = re.sub(r"\u2265", "", prompt_new)
    prompt_new = re.sub(r"\u0131", "i", prompt_new)
    prompt_new = re.sub(r"\u2032", "'", prompt_new)
    prompt_new = re.sub(r"\u03b1", "alpha", prompt_new)

    return prompt_new


# Define the PMID list
excel_path = path_equity_precision_llm_folder + "/llm training/Methods.xlsx"
pmid_list = pd.read_excel(excel_path, sheet_name='Training Data')['PMID'].tolist()
# pmid_list = [22744164, 15561964, 28770629, 33764184, 36155119, 35477306, 23153210, 34794279, 12116231, 2109332, 6453040, 31807629, 28068033, 28668296, 34872002, 32663752, 12475769, 22113416, 12638530, 36649380]  # Replace with PMIDs
# prompt_pmid = prompt_generator(PMID, base_prompts, excel_path)

# Generate loop
for PMID in pmid_list:
    # Generate the customized prompt
    prompt_pmid = prompt_generator(PMID, base_prompts, excel_path)

    # Create a directory for each PMID in the working folder
    pmid_folder = os.path.join(path_equity_precision_llm_folder, "working", str(PMID))
    os.makedirs(pmid_folder, exist_ok=True)


    # Define file paths for each prompt
    file_path_p1 = os.path.join(pmid_folder, f"{base_prompt_files[0]}_{PMID}.txt")
    file_path_p2 = os.path.join(pmid_folder, f"{base_prompt_files[1]}_{PMID}.txt")
    file_path_p3 = os.path.join(pmid_folder, f"{base_prompt_files[2]}_{PMID}.txt")


    # Because we are keeping p1vN, we save the original file
    with open(file_path_p1, "w") as file:
            file.write(base_prompts[0])

    # Because we are keeping p2vN, we save the original file
    with open(file_path_p2, "w") as file:
            file.write(base_prompts[1])

    # Because we are modifying p3vN, we need to save the modified prompt to a new file
    with open(file_path_p3, "w") as file:
            file.write(prompt_pmid)

    print(f"Prompts generated and saved for PMID {PMID}")


22744164
Acculturation*; Asian / psychology; Asian / statistics & numerical data*; Body Mass Index; Diabetes Mellitus, Type 2 / ethnology; Diabetes Mellitus, Type 2 / therapy*; Diet / statistics & numerical data; Female; Glycated Hemoglobin / analysis; Humans; Income / statistics & numerical data; India / ethnology; Male; Middle Aged; Self Care / statistics & numerical data; United States 
Mesh is not null
Prompts generated and saved for PMID 22744164
15561964
Chromosome Mapping*; Diabetes Mellitus, Type 2 / epidemiology; Diabetes Mellitus, Type 2 / genetics*; Diabetic Angiopathies / epidemiology; Female; Genome; Human; Humans; Hypertension / epidemiology; Hypertension / genetics*; Male; Middle Aged; Racial Groups, United States  
Mesh is not null
Prompts generated and saved for PMID 15561964
28770629
Blood Glucose / analysis; Diabetes Mellitus, Type 2 / blood; Diabetes Mellitus, Type 2 / diagnostic imaging; Diabetes Mellitus, Type 2 / drug therapy*; Glucosides / administration & dosag